In [ ]:
#Import packages
%load_ext autoreload
%autoreload 2
import sys
import numpy as np
sys.path.append('./molecular-state-classes-and-functions/')
from classes import UncoupledBasisState, CoupledBasisState, State
from functions import make_hamiltonian, make_hamiltonian_B, make_QN, ni_range, vector_to_state, matrix_to_states
from functions import find_state_idx_from_state
import pickle
from OBE_functions import *
import scipy

### Matrix element functions for OBE integrator ### 
from sympy.physics.wigner import wigner_3j, wigner_6j

def threej_f(j1,j2,j3,m1,m2,m3):
    return complex(wigner_3j(j1,j2,j3,m1,m2,m3))

def sixj_f(j1,j2,j3,j4,j5,j6):
    return complex(wigner_6j(j1,j2,j3,j4,j5,j6))

import matplotlib.pyplot as plt
%matplotlib notebook

## X-state Hamiltonian
Load the X-state hamiltonian from file and transform it to the coupled basis

In [ ]:
H_X_uc = make_hamiltonian("./utilities/TlF_X_state_hamiltonian_J0to4.pickle")

In [ ]:
#Generate lists of quantum numbers
QN_X_uncoupled = make_QN(0,3,1/2,1/2)

Jmin = 0
Jmax = 4
I_F = 1/2
I_Tl = 1/2
QN_X = [CoupledBasisState(F,mF,F1,J,I_F,I_Tl, electronic_state='X', P = (-1)**J, Omega = 0)
      for J  in ni_range(Jmin, Jmax+1)
      for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
      for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
      for mF in ni_range(-F, F+1)
     ]

In [ ]:
### Transform Hamiltonian to coupled basis ###
#Load transform matrix
with open("./utilities/UC_to_C_j0to4.pickle","rb") as f:
    S_trans = pickle.load(f)

In [ ]:
#Transform matrix
E = np.array((0,0,0))
B = np.array((0,0,0.001))
H_X =  S_trans.conj().T @ H_X_uc(E, B) @ S_trans

In [ ]:
D, V = np.linalg.eigh(H_X)

#Diagonalize the Hamiltonian
H_X_diag = V.conj().T @ H_X @ V

#New set of quantum numbers:
QN_X_diag = matrix_to_states(V, QN_X)

state = vector_to_state(V[:,1],QN_X)
state.print_state()

In [ ]:
QN_X_diag[-1].print_state()

Find the part of the Hamiltonian that is used for the simulations

In [ ]:
#Define what states are to be included in the simulation
Js = [0,2]
ground_states = [1*CoupledBasisState(F,mF,F1,J,I_F,I_Tl, electronic_state='X', P = (-1)**J, Omega = 0)
                  for J  in Js
                  for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
                  for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
                  for mF in ni_range(-F, F+1)
                 ]

ground_states_diag = []
for ground_state in ground_states:
    i = find_state_idx_from_state(H_X_diag,ground_state, QN_X_diag)
    ground_states_diag.append(QN_X_diag[i])

In [ ]:
H_X_red = reduced_basis_hamiltonian(QN_X_diag, H_X_diag, ground_states_diag)

In [ ]:
H_X_red.shape

In [ ]:
ground_states_diag[-1].print_state()

## B-state Hamiltonian

In [ ]:
H_B = make_hamiltonian_B("./utilities/B_hamiltonians_symbolic_coupled_P_1to3.pickle")

In [ ]:
Jmin = 1
Jmax = 3
I_F = 1/2
I_Tl = 1/2
Ps = [-1, 1]
QN_B = [CoupledBasisState(F,mF,F1,J,I_F,I_Tl,P = P, Omega = 1, electronic_state='B')
      for J  in ni_range(Jmin, Jmax+1)
      for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
      for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
      for mF in ni_range(-F, F+1)
      for P in Ps
     ]

In [ ]:
D,V = np.linalg.eigh(H_B)

#Diagonalize the Hamiltonian
H_B_diag = V.conj().T @ H_B @ V

#New set of quantum numbers:
QN_B_diag = matrix_to_states(V, QN_B)

state = vector_to_state(V[:,1],QN_B)
state.print_state()

In [ ]:
#Define what states are to be included in the simulation
J = 1
F1 = 3/2
F = 2
excited_states = [1*CoupledBasisState(F,mF,F1,J,I_F,I_Tl, electronic_state='B', P = -1, Omega = 1)
                  for mF in ni_range(-F, F+1)
                 ]

excited_states_diag = []
for excited_state in excited_states:
    i = find_state_idx_from_state(H_B_diag,excited_state, QN_B_diag)
    excited_states_diag.append(QN_B_diag[i])

In [ ]:
H_B_red = reduced_basis_hamiltonian(QN_B_diag, H_B_diag, excited_states_diag)

In [ ]:
H_B_red.shape

## Molecular trajectory
For convenience, defining a function that gives the position of the molecule as a function of time. This makes it easier to convert spatial dependence of laser intensity etc. into time-dependent functions

In [ ]:
def molecule_position(t, r0, v):
    """
    Functions that returns position of molecule at a given time for given initial position and velocity.
    inputs:
    t = time in seconds
    r0 = position of molecule at t = 0 in meters
    v = velocity of molecule in meters per second
    
    returns:
    r = position of molecule in metres
    """
    r =  r0 + v*t
    
    return r

In [ ]:
#Define the position of the molecule as a function of time
r0 = np.array((0,0,-5e-3))
v = np.array((0,0,200))
r_t = lambda t: molecule_position(t, r0, v)

#Define the total time for which the molecule is simulated
z0 = r0[2]
vz = v[2]
T = np.abs(2*z0/vz)

## Optical couplings
Generating the matrix of optical couplings here. Assuming rotating frame so no $\exp(i\omega t)$ time-dependence

In [ ]:
QN = ground_states_diag + excited_states_diag
Js = [0]
ground_states_laser_approx =  [1*CoupledBasisState(F,mF,F1,J,I_F,I_Tl, electronic_state='X', P = (-1)**J, Omega = 0)
                                  for J  in Js
                                  for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
                                  for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
                                  for mF in ni_range(-F, F+1)
                                 ]

ground_states_laser = []
for ground_state in ground_states_laser_approx:
    i = find_state_idx_from_state(H_X_diag,ground_state, QN_X_diag)
    ground_states_laser.append(QN_X_diag[i])

excited_states_laser = excited_states_diag

In [ ]:
H_laser = optical_coupling_matrix(QN, ground_states_laser, excited_states_laser, pol_vec = np.array([0,0,1]), reduced = False)
H_laser[np.abs(H_laser) < 1e-3*np.max(np.abs(H_laser))] = 0

#Check that coupling matrix is hermitian
print(np.allclose(H_laser, H_laser.conj().T))

In [ ]:
#Calculate the matrix element for the "main" transition so that coupling matrix can be scaled to have appropriate rabi rate
#Define approximate form of main ground state
ground_main_approx = 1*CoupledBasisState(J=0,F1=1/2,F=1,mF=0,I1=1/2,I2=1/2,electronic_state='X', P = 1, Omega = 0)
ground_main_i = find_state_idx_from_state(H_X_diag,ground_main_approx, QN_X_diag)
ground_main = QN_X_diag[ground_main_i]

#Define approximate form of main excited state
excited_main_approx = 1*CoupledBasisState(J = 1,F1=3/2,F=2,mF=0,I1=1/2,I2=1/2, electronic_state='B', P = -1, Omega = 1)
excited_main_i = find_state_idx_from_state(H_B_diag,excited_main_approx, QN_B_diag)
excited_main = QN_B_diag[excited_main_i]

ME_main = ED_ME_mixed_state(excited_main, ground_main, pol_vec = np.array([0,0,1]))

print(ME_main)

In [ ]:
ground_main.print_state()

In [ ]:
#Calculate effective dipole moment for the optical transitions
Gamma = 1/100e-9 #Natural linewidth in 2*pi*Hz
f = 3e8/271.7e-9 #Frequency in Hz
D_eff = (np.sqrt(3*np.pi*8.85e-12*1.05e-34*3e8**3*Gamma/(2*np.pi*f)**3)
         /(1/3e8 * 1e-21)* 0.393430307 * 5.291772e-9/4.135667e-15)  #Hz/(V/cm)

#Generate optical coupling matrix with set rabi rate
Omega = 2*np.pi*1000e3*0 #[2pi*Hz]

#Calculate the electric field required to give the desired Rabi rate
laser_power = calculate_power_needed(Omega, ME_main, D_TlF = D_eff, fwhm = 1e-3)
E_laser = lambda t: microwave_field(r_t(t), power=laser_power, fwhm = 1e-3)



H_oc_t = lambda t: D_eff*E_laser(t)*H_laser
# H_oc_t = lambda t: Omega*H_laser/ME_main

In [ ]:
D_eff*E_laser(T/2)*ME_main/(2*np.pi*1e6)

In [ ]:
2*np.pi * 4.2282 * 0.393430307 *5.291772e-9/4.135667e-15

In [ ]:
r_t(T/2)

In [ ]:
laser_power

## Total Hamiltonian

In [ ]:
#Shift the energies of the states to account for transferring to rotating frame
H_X_shifted = np.diag(np.diag(H_X_red) - H_X_diag[ground_main_i,ground_main_i])

detuning = 0
H_B_shifted = np.diag(np.diag(H_B_red - H_B_diag[excited_main_i,excited_main_i] + detuning))

In [ ]:
H_tot_t = lambda t: scipy.linalg.block_diag(H_X_shifted, H_B_shifted) + H_oc_t(t)

In [ ]:
np.diag(H_tot_t(T/2))

## Collapse operators
Generating the matrix representing spontaneous decay from the excited state

In [ ]:
excited_test_approx = 1*CoupledBasisState(J = 1, F1=3/2, F=2, mF=1, I1=1/2, I2=1/2, electronic_state='B', P = -1, Omega = 1)
excited_test_i = find_state_idx_from_state(H_B_diag,excited_test_approx, QN_B_diag)
excited_test = QN_B_diag[excited_test_i]

ground_test_approx = 1*CoupledBasisState(J=0,F1=1/2,F=1,mF=-1,I1=1/2,I2=1/2,electronic_state='X', P = 1, Omega = 0)
ground_test_i = find_state_idx_from_state(H_X_diag,ground_test_approx, QN_X_diag)
ground_test = QN_X_diag[ground_test_i]

BRs = calculate_BR(excited_test, ground_states)
excited_test.print_state()

In [ ]:
# for ground_state, BR in zip(ground_states, BRs):
#     if BR > 0.001:
#         print("Branching ratio to")
#         ground_state.print_state()
#         print("is {:5f}\n".format(BR))

In [ ]:
Gamma = 1*2*np.pi*1.6e6 #Natural linewidth in 2pi*Hz

C_list = collapse_matrices(QN, ground_states_diag, excited_states_diag, gamma = Gamma)

In [ ]:
len(C_list)

## Density matrix

In [ ]:
# Define states that are populated initially
# Js = [0]
# Fs = [1]
# states_pop_approx = [1*CoupledBasisState(F,mF,F1,J,I_F,I_Tl, electronic_state='X', P = (-1)**J, Omega = 0)
#               for J  in Js
#               for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
#               for F in Fs
#               for mF in ni_range(-F, F+1)
#              ]

# states_pop = []
# for state in states_pop_approx:
#     i = find_state_idx_from_state(H_X_diag,state, QN_X_diag)
#     states_pop.append(QN_X_diag[i])

Js = [1]
Fs = [2]
F1s = [3/2]
mFs = [-2]
states_pop_approx = [1*CoupledBasisState(F,mF,F1,J,I_F,I_Tl, electronic_state='B', P = -1, Omega = 1)
              for J  in Js
              for F1 in F1s
              for F in Fs
              for mF in mFs
             ]

states_pop = []
for state in states_pop_approx:
    i = find_state_idx_from_state(H_B_diag,state, QN_B_diag)
    states_pop.append(QN_B_diag[i])

pops = np.ones(len(states_pop))/len(states_pop)

rho_ini = generate_density_matrix(QN,states_pop,pops)

## Transferring to superoperator basis

In [ ]:
#Calculate Liouvillian
#First compute the part that contains the spontaneous decay
L_collapse = np.zeros((len(QN)**2,len(QN)**2), dtype = complex)
for C in C_list:
    L_collapse += (generate_superoperator(C,C.conj().T)
                     -1/2 * (generate_flat_superoperator(C.conj().T @ C) + generate_sharp_superoperator(C.conj().T @ C)))

L_t = lambda t: (-1j*generate_commutator_superoperator(H_tot_t(t)) + L_collapse)

In [ ]:
len(L_collapse[L_collapse != 0])

In [ ]:
L_test = csc_matrix(L_t(t_i)*dt)
expm_sparse(L_test)

## Time-evolving

In [ ]:
from scipy.sparse.linalg import expm as expm_sparse
from scipy.sparse import csc_matrix, csr_matrix, lil_matrix

#Generate rho vector
rho_vec = generate_rho_vector(rho_ini)

T = 3e-6

Nsteps = int(10e1) #Number of timesteps
dt = T/Nsteps

#Generate array of times
t_array = np.linspace(0,T,Nsteps)

#Array for storing results
pop_results = np.zeros((len(QN), len(t_array)), dtype = float)

pop_results[:,0] = np.diag(rho_ini)

#Loop over timesteps
for i, t_i in enumerate(tqdm(t_array[1:])):
#     rho_vec = expm(L_t(t_i) * dt) @ rho_vec
    L_sparse = csc_matrix(L_t(t_i)*dt)
    #rho_vec = expm_sparse(L_sparse) @ rho_vec 
    
    rho_vec = expm_multiply(L_sparse, rho_vec) 
    
    rho = rho_vec.reshape(len(QN),len(QN))
    pop_results[:,i+1] = np.real(np.diag(rho))
    
#     if not np.allclose(rho - np.diag(rho),np.zeros(rho.shape)):
#         print("rho not diagonal {}".format(i))
#         print(rho[25,26])
    
    

In [ ]:
fig, ax = plt.subplots()
ax.plot(t_array*1e6, pop_results.T)

In [ ]:
#Plot populations in different J
P0 = np.sum(pop_results[0:4,:], axis = 0)
P2 = np.sum(pop_results[4:24,:], axis = 0)
PB1 = np.sum(pop_results[24:,:], axis = 0)

fig, ax = plt.subplots()
ax.plot(t_array*1e6, P0, label = 'X, J = 0')
ax.plot(t_array*1e6, P2, label = 'X, J = 2')
ax.plot(t_array*1e6, PB1, label = 'B, J = 1')
ax.legend()
ax.set_xlabel("Time / us")
ax.set_ylabel("Population in state")

## Scattered photons
Calculating number of scattered photons

In [ ]:
N_photons = np.sum(PB1*dt*Gamma/(2*np.pi))
print(N_photons)

## Speed tests

In [ ]:
import timeit
from scipy.linalg import expm
L = L_t(T/2)
timeit.timeit('M = expm(L*dt) @ rho_vec', number = 10, globals = globals())

In [ ]:
from scipy.sparse import csc_matrix, csr_matrix 
from scipy.sparse.linalg import expm_multiply, expm


L_sparse = csc_matrix(L_t(T/2)*dt)

In [ ]:
timeit.timeit('L_sparse = csc_matrix(L*dt); M = expm_multiply(L_sparse,rho_vec)', number = 10, globals = globals())

In [ ]:
timeit.timeit('L_sparse = csc_matrix(L*dt*dt); M = expm(L_sparse) @ rho_vec', number = 10, globals = globals())

In [ ]:
L_sparse

In [ ]:
dt